# Import Modules

In [ ]:
import os
print(os.getcwd())

In [ ]:
%%capture

import sys
import time

import itertools

import numpy as np


from multiprocessing import Pool
from functools import partial

# #############################################################################
from methods import run_al_i
from misc_modules.misc_methods import GetFriendlyID

In [ ]:
t0 = time.time()

In [ ]:
from inputs import (
    stoich_i,
    verbose,
    gp_settings,
    runs_list,
    acquisition_methods,
    duplicate_analysis,
    )

In [ ]:
variables_dict = dict(
    stoich_i=stoich_i,
    # acquisition_method=acquisition_method,
    # duplicate_analysis=duplicate_analysis,
    verbose=verbose,
    gp_settings=gp_settings,
    )

# def run_al_meth(
#     input_dict,
#     stoich_i=None,
#     # acquisition_method=None,
#     verbose=None,
#     gp_settings=None,
#     name_i=None,
#     save_dir=None,
#     # duplicate_analysis=None,
#     ):
#     """
#     """
#     i = input_dict['i']
#     acquisition_method = input_dict['acquisition_method']
#     duplicate_analysis = input_dict['duplicate_analysis' ]
#     seed = input_dict['seed']

#     # #########################################################################
#     i_str = str(i).zfill(2)

    
#     print(80 * "#")
#     print(i_str, 77 * "#")

#     # name_i = "TEST_AL_7_" + GetFriendlyID()
#     name_i = "TEST_AL_6_" + GetFriendlyID()

#     print("****")
#     print("name_i: ", name_i)
#     print("****")

#     save_dir = stoich_i + "/" + acquisition_method + "_" + str(duplicate_analysis)
#     run_al_i(
#         stoich_i=stoich_i,
#         verbose=verbose,
#         gp_settings=gp_settings,
#         name_i=name_i,
#         save_dir_extra=save_dir,
#         acquisition_method=acquisition_method,
#         duplicate_analysis=duplicate_analysis,
#         seed=seed,
#         )

In [ ]:
input_list = []
for i in itertools.product(acquisition_methods, duplicate_analysis, runs_list):
    data_dict_i = dict(
        acquisition_method=i[0],
        duplicate_analysis=i[1],
        i=i[2],
        seed=np.random.randint(0, 1000)
        )
    input_list.append(data_dict_i)
    
# input_list

In [ ]:
input_dict = input_list[0]

In [ ]:
# def run_al_meth(

# input_dict,
# stoich_i=None
# # acquisition_method=None,
# verbose=None
# gp_settings=None
# name_i=None
# save_dir=None

# duplicate_analysis=None,

In [ ]:
# def run_al_meth(
# input_dict,
# stoich_i=None,
# # acquisition_method=None,
# verbose=None,
# gp_settings=None,
# name_i=None,
# save_dir=None,
# # duplicate_analysis=None,
# ):
"""
"""
i = input_dict['i']
acquisition_method = input_dict['acquisition_method']
duplicate_analysis = input_dict['duplicate_analysis' ]
seed = input_dict['seed']

# #########################################################################
i_str = str(i).zfill(2)


print(80 * "#")
print(i_str, 77 * "#")

# name_i = "TEST_AL_7_" + GetFriendlyID()
name_i = "TEST_AL_6_" + GetFriendlyID()

print("****")
print("name_i: ", name_i)
print("****")

save_dir = stoich_i + "/" + acquisition_method + "_" + str(duplicate_analysis)

# run_al_i(
#     stoich_i=stoich_i,
#     verbose=verbose,
#     gp_settings=gp_settings,
#     name_i=name_i,
#     save_dir_extra=save_dir,
#     acquisition_method=acquisition_method,
#     duplicate_analysis=duplicate_analysis,
#     seed=seed,
#     )

In [ ]:
# def run_al_i(
#     stoich_i=None,
#     verbose=None,
#     gp_settings=None,
#     name_i=None,
#     save_dir_extra=None,
#     acquisition_method=None,
#     duplicate_analysis=None,
#     seed=None,
#     ):

"""
"""
# | - run_al_i
# # Read Data

# + {"jupyter": {"source_hidden": true}}
sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling"))
from ml_methods import get_data_for_al

out_dict = get_data_for_al(
    stoich=stoich_i, verbose=False,
    drop_too_many_atoms=True,
#     drop_too_many_atoms=False,
    )

df_bulk_dft = out_dict["df_bulk_dft"]
print("df_bulk_dft.shape:", df_bulk_dft.shape)

df_bulk_dft = df_bulk_dft[df_bulk_dft["source"] == "raul"]
print("df_bulk_dft.shape:", df_bulk_dft.shape)

# df_bulk_dft = df_bulk_dft[["atoms", "energy_pa"]]
df_bulk_dft = df_bulk_dft[["atoms", "dH"]]
df_bulk_dft.columns.values[1] = "y_real"

df_features_pre = out_dict["df_features_pre"]
df_features_post = out_dict["df_features_post"]

df_ids = out_dict["df_ids"]


df_static_irox = out_dict["df_static_irox"]
df_dij = out_dict["df_dij"]
# -

# # Filter to candidates w/ DFT energy

# + {"jupyter": {"source_hidden": true}}
ids_w_dft = df_bulk_dft.index

# TEMP | Reduce size of candidate space
# np.random.seed(8)
# ids_w_dft = np.sort(np.random.choice(np.sort(ids_w_dft), size=200))
ids_w_dft = list(set(ids_w_dft))
# print("ids_w_dft:", ids_w_dft)

df_bulk_dft = df_bulk_dft.loc[ids_w_dft]
print("df_bulk_dft.shape:", df_bulk_dft.shape)

df_features_pre = df_features_pre.loc[ids_w_dft]
df_features_post = df_features_post.loc[ids_w_dft]

In [ ]:
df_bulk_dft.index.unique()

In [ ]:
# + {"active": ""}
#
#
#
#
# -

# # CCF Class

#  # + {"jupyter": {"source_hidden": true}}
#  sys.path.insert(0, os.path.join(
#      os.environ["PROJ_irox"],
#      "python_classes"))
#  from ccf_similarity.ccf import CCF
#
#  d_thresh = 0.02
#  CCF = CCF(
#      df_dij=df_dij,
#      d_thresh=d_thresh)
#
#  # + {"jupyter": {"source_hidden": true}}
#  RM = RegressionModel(
#      opt_hyperparameters=True,
#      gp_settings_dict=gp_settings,
#      verbose=verbose,
#      )
#
#  FP = FingerPrints(
#      df_features_pre,
#      df_features_post=df_features_post,
#      pca_mode="num_comp",  # 'num_comp' or 'perc'
#      pca_comp=10,
#      pca_perc=None,
#      verbose=verbose,
#      )
#
#  CS = CandidateSpace(
#      Y_data=df_bulk_dft,
#      Y_key="y_real",
#      FingerPrints=FP,
#      )
#
#  # +
#  AL = ALBulkOpt(
#      CandidateSpace=CS,
#      RegressionModel=RM,
#      DuplicateFinder=CCF,  # Optional
#      duplicate_analysis=duplicate_analysis,
#      # num_seed_calcs=11,
#      num_seed_calcs=5,
#      acquisition_bin=5,
#      # stop_mode="num_generations",
#      stop_mode=None,
#      stop_num_generations=3,
#      name=name_i,
#      save_dir_extra=save_dir_extra,
#      verbose=verbose,
#      # acquisition_method="gp_ucb",
#      acquisition_method=acquisition_method,
#      seed=seed,
#      )
#
#  run_al = True
#  if run_al:
#      AL.run_AL()
#      AL.duplicate_system_history_analysis()
#      AL.__save_state__()

#__|

In [ ]:
# for i in range(num_runs):
#     run_al_meth(i, **variables_dict)

In [ ]:
# traces_all = Pool().map(
#     partial(
#         run_al_meth,  # METHOD
#         **variables_dict,  # KWARGS
#         ),
#     input_list,
#     )

In [ ]:
print("Notebook runtime (s):", time.time() - t0)
print("Notebook runtime (min):", (time.time() - t0) / 60)

In [ ]:
# stoich_i = "AB3"
# verbose = False

# gp_settings = {
#     "noise": 0.02542,
#     "sigma_l": 1.0049,
#     "sigma_f": 5.19,
#     "alpha": 0.018,
#     }

# # duplicate_analysis = False
# # acquisition_method = "gp_ucb"
# # acquisition_method = "random"

# # #############################################################################
# runs_list = list(range(5))
# acquisition_methods = ["gp_ucb", "random"]
# duplicate_analysis = [True, False]

# # TEST SETTINGS # #############################################################
# runs_list = list(range(1))
# acquisition_methods = ["gp_ucb"]
# duplicate_analysis = [True]